## Initialization

In [1]:
import pandas as pd
df = pd.read_csv('../../FeatureEngineering/MetaData/data6_&_odds.csv')
df.dropna(inplace=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, SelectFromModel, mutual_info_classif
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding
from keras.models import Sequential


import numpy as np

import time

In [14]:
train_data = df.loc[(df.season <= 2013) & (df.season >= 2007)]
valid_data = df.loc[(df.season > 2013) & (df.season < 2016)]
test_data = df.loc[df.season == 2016]
full_train_data = pd.concat([train_data, valid_data], axis=0)

X_train, y_train = train_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), train_data.home_team_wins
X_val, y_val = valid_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), valid_data.home_team_wins
X_test, y_test = test_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), test_data.home_team_wins


## GNB Testing per Season

In [15]:
X_train_sfm = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_sfm = y_train

X_test_sfm = test_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_test_sfm = y_test

win_accuracy = {}

In [16]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
# κάνουμε εκπαίδευση (fit) δηλαδή ουσιαστικά υπολογίζουμε μέση τιμή και διακύμανση για όλα τα χαρακτηριστικά και κλάσεις στο training set
model = gnb.fit(X_train_sfm, y_train_sfm)
# η GaussianNB έχει builtin μέθοδο υπολογισμό accuracy. Αποθηκεύουμε την τιμή της στον πίνακά μας με τα αποτελέσματα από τα άλλα classifiers
win_accuracy['gaussian naive bayes'] = gnb.score(X_test_sfm, y_test_sfm)
# και ξανατυπώνουμε τα sorted αποτελέσματα
print("Classification Accuracy on the NBA Games Dataset (40% test set)\n")
sorted_accuracy = [(k, win_accuracy[k]) for k in sorted(win_accuracy, key=win_accuracy.get, reverse=True)]
for k, v in sorted_accuracy:
  print(k,v)

Classification Accuracy on the NBA Games Dataset (40% test set)

gaussian naive bayes 0.6838235294117647


## kNN Testing per Season

In [14]:
X_train_rfe = train_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'W_PCT_home',
            'HOME_RECORD_home', 'ROAD_RECORD_home', 'ROAD_RECORD_away',
            'WIN_PRCT_home_3g', 'PTS_home_3g', 'REB_home_3g', 'WIN_PRCT_away_3g',
            'AST_away_3g', 'REB_away_3g', 'PTS_home_7g', 'AST_home_7g',
            'REB_home_7g', 'AST_away_7g', 'diff_avg_pts_home', 'diff_avg_ast_home',
            'diff_avg_ast_away', 'diff_avg_fg3_pct_home', 'diff_avg_reb_home',
            'diff_avg_reb_away', 'top_players', 'eff_visitor', 'HG_7days',
            'AG_7days', 'G_7days', 'back2back', 'HG_7days_VISITOR',
            'G_7days_VISITOR', 'back2back_visitor', 'missing_players',
            'missing_players_visitor', 'home_elo', 'visitor_elo', 'elo_diff',
            'top_player_diff', 'missing_player_diff', 'eff_diff', 'month',
            'Home_Last_5_Avg_AST_home', 'Home_Last_5_Avg_REB_home',
            'Home_Last_5_Avg_PTS_away', 'Home_Last_5_Avg_REB_away',
            'Home_Last_5_Avg_AST_away', 'Away_Last_5_Avg_FG3_PCT_home',
            'Away_Last_5_Avg_AST_home', 'Away_Last_5_Avg_PTS_away',
            'Away_Last_5_Avg_FT_PCT_away', 'Away_Last_5_Avg_AST_away',
            'diff_ast_last_3_games', 'diff_ast_last_7_games',
            'diff_reb_last_3_games', 'diff_reb_last_7_games',
            'diff_win_pct_3_last_games', 'diff_curr_win_pct',
            'diff_curr_home_record', 'diff_curr_away_record']]

y_train_rfe = y_train

X_val_rfe = valid_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'W_PCT_home',
            'HOME_RECORD_home', 'ROAD_RECORD_home', 'ROAD_RECORD_away',
            'WIN_PRCT_home_3g', 'PTS_home_3g', 'REB_home_3g', 'WIN_PRCT_away_3g',
            'AST_away_3g', 'REB_away_3g', 'PTS_home_7g', 'AST_home_7g',
            'REB_home_7g', 'AST_away_7g', 'diff_avg_pts_home', 'diff_avg_ast_home',
            'diff_avg_ast_away', 'diff_avg_fg3_pct_home', 'diff_avg_reb_home',
            'diff_avg_reb_away', 'top_players', 'eff_visitor', 'HG_7days',
            'AG_7days', 'G_7days', 'back2back', 'HG_7days_VISITOR',
            'G_7days_VISITOR', 'back2back_visitor', 'missing_players',
            'missing_players_visitor', 'home_elo', 'visitor_elo', 'elo_diff',
            'top_player_diff', 'missing_player_diff', 'eff_diff', 'month',
            'Home_Last_5_Avg_AST_home', 'Home_Last_5_Avg_REB_home',
            'Home_Last_5_Avg_PTS_away', 'Home_Last_5_Avg_REB_away',
            'Home_Last_5_Avg_AST_away', 'Away_Last_5_Avg_FG3_PCT_home',
            'Away_Last_5_Avg_AST_home', 'Away_Last_5_Avg_PTS_away',
            'Away_Last_5_Avg_FT_PCT_away', 'Away_Last_5_Avg_AST_away',
            'diff_ast_last_3_games', 'diff_ast_last_7_games',
            'diff_reb_last_3_games', 'diff_reb_last_7_games',
            'diff_win_pct_3_last_games', 'diff_curr_win_pct',
            'diff_curr_home_record', 'diff_curr_away_record']]

y_val_rfe = y_val

X_test_rfe = test_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'W_PCT_home',
            'HOME_RECORD_home', 'ROAD_RECORD_home', 'ROAD_RECORD_away',
            'WIN_PRCT_home_3g', 'PTS_home_3g', 'REB_home_3g', 'WIN_PRCT_away_3g',
            'AST_away_3g', 'REB_away_3g', 'PTS_home_7g', 'AST_home_7g',
            'REB_home_7g', 'AST_away_7g', 'diff_avg_pts_home', 'diff_avg_ast_home',
            'diff_avg_ast_away', 'diff_avg_fg3_pct_home', 'diff_avg_reb_home',
            'diff_avg_reb_away', 'top_players', 'eff_visitor', 'HG_7days',
            'AG_7days', 'G_7days', 'back2back', 'HG_7days_VISITOR',
            'G_7days_VISITOR', 'back2back_visitor', 'missing_players',
            'missing_players_visitor', 'home_elo', 'visitor_elo', 'elo_diff',
            'top_player_diff', 'missing_player_diff', 'eff_diff', 'month',
            'Home_Last_5_Avg_AST_home', 'Home_Last_5_Avg_REB_home',
            'Home_Last_5_Avg_PTS_away', 'Home_Last_5_Avg_REB_away',
            'Home_Last_5_Avg_AST_away', 'Away_Last_5_Avg_FG3_PCT_home',
            'Away_Last_5_Avg_AST_home', 'Away_Last_5_Avg_PTS_away',
            'Away_Last_5_Avg_FT_PCT_away', 'Away_Last_5_Avg_AST_away',
            'diff_ast_last_3_games', 'diff_ast_last_7_games',
            'diff_reb_last_3_games', 'diff_reb_last_7_games',
            'diff_win_pct_3_last_games', 'diff_curr_win_pct',
            'diff_curr_home_record', 'diff_curr_away_record']]

y_test_rfe = y_test

In [15]:
import time

# Split Data to Train and Validation

clf = KNeighborsClassifier()
# η παράμετρος n_jobs = 1 χρησιμοποιεί όλους τους πυρήνες του υπολογιστή
params = {'n_neighbors': np.arange(1, 15),
          'leaf_size': list(range(1, 5, 1))}

estimator = GridSearchCV(clf, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
start_time = time.time()
estimator.fit(X_train_rfe, y_train_rfe)
val_score = estimator.score(X_val_rfe, y_val_rfe)

preds = estimator.predict(X_test_rfe)
test_score = estimator.score(X_test_rfe, y_test_rfe)

target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_rfe, preds, target_names=target_names))
print("val score:", val_score)
print(estimator.best_estimator_)
print(estimator.best_params_)
print("best score:", estimator.best_score_)
print("test score", test_score)

Συνολικός χρόνος fit και predict: 35.21475648880005 seconds
              precision    recall  f1-score   support

   home_loss       0.62      0.51      0.56       429
    home_win       0.65      0.75      0.70       523

    accuracy                           0.64       952
   macro avg       0.64      0.63      0.63       952
weighted avg       0.64      0.64      0.64       952

val score: 0.6661157024793388
KNeighborsClassifier(leaf_size=1, n_neighbors=11)
{'leaf_size': 1, 'n_neighbors': 11}
best score: 0.6560869565217391
test score 0.6407563025210085


## MLP Testing per Season

In [17]:
X_train_uni = train_data[['diff_curr_win_pct','diff_curr_away_record','odds_home','odds_away','elo_diff']]
y_train_uni = y_train

X_val_uni = valid_data[['diff_curr_win_pct','diff_curr_away_record','odds_home','odds_away','elo_diff']]
y_val_uni = y_val

X_test_uni = test_data[['diff_curr_win_pct','diff_curr_away_record','odds_home','odds_away','elo_diff']]
y_test_uni = y_test

In [18]:
# Multilayer Perceptron
model = MLPClassifier()
model.fit(X_train_uni, y_train_uni)

# defining parameter range
param_grid = {'max_iter': list(range(100, 300, 50)),
              # 'activation':['identity', 'logistic', 'tanh', 'relu'],
              'hidden_layer_sizes': [(20, 10, 5)],
              'solver': ['lbfgs', 'sgd'],
              'alpha': [1e-05],
              }


grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)


In [19]:
start_time = time.time()
grid.fit(X_train_uni, y_train_uni)
val_score = grid.score(X_val_uni, y_val_uni)

preds = grid.predict(X_test_uni)
test_score = grid.score(X_test_uni, y_test_uni)

target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_uni, preds, target_names=target_names))
print("val score:", val_score)

model = grid.best_estimator_
y_fit = model.predict(X_test_uni)

print(grid.best_params_)
print("best score:", grid.best_score_)
print("test score", test_score)


Συνολικός χρόνος fit και predict: 21.410220861434937 seconds
              precision    recall  f1-score   support

   home_loss       0.68      0.58      0.63       429
    home_win       0.69      0.77      0.73       523

    accuracy                           0.69       952
   macro avg       0.69      0.68      0.68       952
weighted avg       0.69      0.69      0.68       952

val score: 0.6995867768595041
{'alpha': 1e-05, 'hidden_layer_sizes': (20, 10, 5), 'max_iter': 100, 'solver': 'lbfgs'}
best score: 0.6908695652173912
test score 0.6869747899159664


## SVM Testing per Season

In [37]:
X_train_rcv = train_data[['missing_player_diff','odds_home','odds_away','visitor_elo']]
y_train_rcv = y_train

X_test_rcv = test_data[['missing_player_diff','odds_home','odds_away','visitor_elo']]
y_test_rcv = y_test

In [38]:
# Support vector classifier
model = SVC()
model.fit(X_train_rcv, y_train_rcv)

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1)

In [39]:
start_time = time.time()
grid.fit(X_train_uni, y_train_uni)

preds = grid.predict(X_test_uni)
test_score = grid.score(X_test_uni, y_test_uni)

target_names = ['home_loss', 'home_win']


print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_uni, preds, target_names=target_names))

model = grid.best_estimator_
y_fit = model.predict(X_test_uni)

print(grid.best_params_)
print("best score:", grid.best_score_)
print("test score", test_score)


Συνολικός χρόνος fit και predict: 277.7286784648895 seconds
              precision    recall  f1-score   support

   home_loss       0.66      0.52      0.58       429
    home_win       0.66      0.78      0.72       523

    accuracy                           0.66       952
   macro avg       0.66      0.65      0.65       952
weighted avg       0.66      0.66      0.65       952

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
best score: 0.6857971014492754
test score 0.6607142857142857


## Random Forest Testing per Season

In [55]:
X_sfm = train_data[['odds_home', 'odds_away', 'W_PCT_home', 'HOME_RECORD_home',
       'ROAD_RECORD_home', 'W_PCT_away', 'HOME_RECORD_away',
       'ROAD_RECORD_away', 'FT_PCT_home_3g', 'FG3_PCT_home_3g',
       'FT_PCT_away_3g', 'FT_PCT_home_7g', 'FT_PCT_away_7g',
       'diff_avg_fg3_pct_home', 'diff_avg_fg3_pct_away',
       'diff_avg_ft_pct_home', 'diff_avg_ft_pct_away', 'diff_avg_reb_away',
       'eff', 'eff_visitor', 'home_elo', 'visitor_elo', 'elo_diff', 'eff_diff',
       'Home_Last_5_Avg_FT_PCT_home', 'Home_Last_5_Avg_FG3_PCT_home',
       'Home_Last_5_Avg_FG_PCT_away', 'Home_Last_5_Avg_FT_PCT_away',
       'Home_Last_5_Avg_FG3_PCT_away', 'Away_Last_5_Avg_FG3_PCT_home',
       'Away_Last_5_Avg_FG_PCT_away', 'Away_Last_5_Avg_FT_PCT_away',
       'Away_Last_5_Avg_FG3_PCT_away', 'diff_fg_pct_last_7_games',
       'diff_ft_pct_last_3_games', 'diff_ft_pct_last_7_games',
       'diff_win_pct_prev_season', 'diff_home_record_last_season',
       'diff_road_record_last_season', 'diff_curr_win_pct',
       'diff_curr_home_record', 'diff_curr_away_record']]
y_sfm = y_train


X_test_sfm = test_data[['odds_home', 'odds_away', 'W_PCT_home', 'HOME_RECORD_home',
       'ROAD_RECORD_home', 'W_PCT_away', 'HOME_RECORD_away',
       'ROAD_RECORD_away', 'FT_PCT_home_3g', 'FG3_PCT_home_3g',
       'FT_PCT_away_3g', 'FT_PCT_home_7g', 'FT_PCT_away_7g',
       'diff_avg_fg3_pct_home', 'diff_avg_fg3_pct_away',
       'diff_avg_ft_pct_home', 'diff_avg_ft_pct_away', 'diff_avg_reb_away',
       'eff', 'eff_visitor', 'home_elo', 'visitor_elo', 'elo_diff', 'eff_diff',
       'Home_Last_5_Avg_FT_PCT_home', 'Home_Last_5_Avg_FG3_PCT_home',
       'Home_Last_5_Avg_FG_PCT_away', 'Home_Last_5_Avg_FT_PCT_away',
       'Home_Last_5_Avg_FG3_PCT_away', 'Away_Last_5_Avg_FG3_PCT_home',
       'Away_Last_5_Avg_FG_PCT_away', 'Away_Last_5_Avg_FT_PCT_away',
       'Away_Last_5_Avg_FG3_PCT_away', 'diff_fg_pct_last_7_games',
       'diff_ft_pct_last_3_games', 'diff_ft_pct_last_7_games',
       'diff_win_pct_prev_season', 'diff_home_record_last_season',
       'diff_road_record_last_season', 'diff_curr_win_pct',
       'diff_curr_home_record', 'diff_curr_away_record']]
y_test_sfm = y_test

# Split our data
X_train_sfm, y_train_sfm = X_sfm, y_sfm


In [56]:
# Random Forest Classifier
clf = RandomForestClassifier()

# defining parameter range
param_grid = {'bootstrap': [True],
              'max_depth': [8, 9, 10, 11],
              'max_features': ['auto', 'sqrt', 'log2'],
              'min_samples_leaf': [3, 4, 5],
              'min_samples_split': [8, 10, 12],
              'n_estimators': [100]}

grid = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
start_time = time.time()
grid.fit(X_train_sfm, y_train_sfm)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [8, 9, 10, 11],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100]},
             scoring='accuracy')

In [57]:
# Predict
preds = grid.predict(X_test_sfm)
test_score = grid.score(X_test_sfm, y_test_sfm)

target_names=['home loss', 'home win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_sfm, preds, target_names=target_names))


model = grid.best_estimator_
y_fit = model.predict(X_test_sfm)

print(grid.best_params_)
print("best score:", grid.best_score_)
print("test score", test_score)

Συνολικός χρόνος fit και predict: 209.17820286750793 seconds
              precision    recall  f1-score   support

   home loss       0.67      0.57      0.62       429
    home win       0.69      0.77      0.73       523

    accuracy                           0.68       952
   macro avg       0.68      0.67      0.67       952
weighted avg       0.68      0.68      0.68       952

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 100}
best score: 0.6949275362318841
test score 0.6827731092436975


## XGBoost Testing per Season

In [74]:
X_train_lasso = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_lasso = y_train


X_test_lasso = test_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]

y_test_lasso = y_test

In [75]:
# XG Boost classifier
model = xgb.XGBClassifier()
model.fit(X_train_lasso, y_train_lasso)

# defining parameter range
param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.3],
        'n_estimators': [100, 200, 300],
        'min_child_weight': [1, 3],
        'gamma': [x/10 for x in range(0, 5)]
        }
grid = GridSearchCV(model, param_grid, scoring='accuracy', n_jobs=-1)

start_time = time.time()
grid.fit(X_train_lasso, y_train_lasso)

preds = grid.predict(X_test_lasso)
test_score = grid.score(X_test_lasso, y_test_lasso)



In [76]:
target_names = ['home_loss', 'home_win']


print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_lasso, preds, target_names=target_names))

model = grid.best_estimator_
y_fit = model.predict(X_test_lasso)

print(grid.best_params_)
print("best score:", grid.best_score_)
print("test score", test_score)


Συνολικός χρόνος fit και predict: 362.4185836315155 seconds
              precision    recall  f1-score   support

   home_loss       0.63      0.55      0.59       495
    home_win       0.72      0.78      0.75       716

    accuracy                           0.69      1211
   macro avg       0.67      0.66      0.67      1211
weighted avg       0.68      0.69      0.68      1211

{'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100}
best score: 0.6914492753623189
test score 0.685383980181668


## 2-Stage Stacking Testing per Season

In [91]:
X_train_lasso = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_lasso = y_train


X_test_lasso = test_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_test_lasso = y_test

In [92]:
clf = [ ('svm', SVC(random_state=42, C=100, gamma=0.001, kernel='rbf')),
        ('xgb', xgb.XGBClassifier(gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=3, n_estimators=100)),
        # ('knn', KNeighborsClassifier(leaf_size=1, n_neighbors=11)),
        # ('mlp', MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 10, 5), max_iter=150, solver='lbfgs')),
        ('rf', RandomForestClassifier(bootstrap=True, max_depth=11, max_features='auto', min_samples_leaf=4, min_samples_split=8, n_estimators=100)),
        ('gnb', GaussianNB())]


ada = AdaBoostClassifier(learning_rate=0.1, n_estimators=100)
mlp = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 10, 5), max_iter=150, solver='lbfgs')

stacking_model = StackingClassifier(estimators=clf, final_estimator=mlp, stack_method='auto', n_jobs=-1)

In [93]:
start_time = time.time()
stacking_model.fit(X_train_lasso, y_train_lasso)

preds = stacking_model.predict(X_test_lasso)
test_score = stacking_model.score(X_test_lasso, y_test_lasso)

target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_lasso, preds, target_names=target_names))
print("test score:", test_score)


Συνολικός χρόνος fit και predict: 15.292037963867188 seconds
              precision    recall  f1-score   support

   home_loss       0.68      0.59      0.63       429
    home_win       0.70      0.78      0.73       523

    accuracy                           0.69       952
   macro avg       0.69      0.68      0.68       952
weighted avg       0.69      0.69      0.69       952

test score: 0.6901260504201681


## 3-Stage Stacking Testing per Season

In [108]:
X_train_lasso = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_lasso = y_train


X_test_lasso = test_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_test_lasso = y_test

In [109]:
layer_one_estimators = [ 
        ('svm', SVC(random_state=42, C=100, gamma=0.001, kernel='rbf')),
        ('rf', RandomForestClassifier(bootstrap=True, max_depth=11, max_features='auto', min_samples_leaf=4, min_samples_split=8, n_estimators=100)),
        # ('xgb', xgb.XGBClassifier(gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=3, n_estimators=100))
        ]   
        # ('knn', KNeighborsClassifier(leaf_size=1, n_neighbors=11)),
        # ('mlp', MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 10, 5), max_iter=150, solver='lbfgs')),
layer_two_estimators = [
        ('xgb', xgb.XGBClassifier(gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=3, n_estimators=100)),
        # ('rf', RandomForestClassifier(bootstrap=True, max_depth=11, max_features='auto', min_samples_leaf=4, min_samples_split=8, n_estimators=100)),
        ('gnb', GaussianNB())]

ada = AdaBoostClassifier(learning_rate=0.1, n_estimators=100)
mlp = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 10, 5), max_iter=150, solver='lbfgs')


layer_two = StackingClassifier(estimators=layer_one_estimators, final_estimator=ada)

# make predictions with the 3-stage stacking model

stacking_model = StackingClassifier(estimators=layer_two_estimators, final_estimator=layer_two) 

In [110]:
start_time = time.time()
stacking_model.fit(X_train_lasso, y_train_lasso)

preds = stacking_model.predict(X_test_lasso)
test_score = stacking_model.score(X_test_lasso, y_test_lasso)

target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_lasso, preds, target_names=target_names))
print("test score:", test_score)


Συνολικός χρόνος fit και predict: 16.13227915763855 seconds
              precision    recall  f1-score   support

   home_loss       0.66      0.60      0.63       429
    home_win       0.69      0.74      0.72       523

    accuracy                           0.68       952
   macro avg       0.68      0.67      0.67       952
weighted avg       0.68      0.68      0.68       952

test score: 0.6785714285714286


## Voting Testing per Season

In [126]:
X_train_lasso = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_lasso = y_train

X_test_lasso = test_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_test_lasso = y_test

In [127]:
# create a voting classifier with hard voting
voting_classifier_hard = VotingClassifier(
    estimators = [('svm', SVC(random_state=42, C=100, gamma=0.001, kernel='rbf')),
                  ('xgb', xgb.XGBClassifier(gamma=0.2, learning_rate=0.1, max_depth=3, min_child_weight=3, n_estimators=100)),
                  ('knn', KNeighborsClassifier(leaf_size=1, n_neighbors=11)),
                  ('mlp', MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 10, 5), max_iter=150, solver='lbfgs')),
                  ('rf', RandomForestClassifier(bootstrap=True, max_depth=11, max_features='auto', min_samples_leaf=4, min_samples_split=8, n_estimators=100)),
                  ('gnb', GaussianNB())],
    voting='hard')


# make predictions with the hard voting model
voting_classifier_hard.fit(X_train_lasso, y_train_lasso)
y_pred_vch = voting_classifier_hard.predict(X_test_lasso)

In [128]:
test_score = voting_classifier_hard.score(X_test_lasso, y_test_lasso)

target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test_lasso, y_pred_vch, target_names=target_names))
print("test score:", test_score)

Συνολικός χρόνος fit και predict: 305.386212348938 seconds
              precision    recall  f1-score   support

   home_loss       0.60      0.54      0.57       502
    home_win       0.70      0.75      0.72       707

    accuracy                           0.66      1209
   macro avg       0.65      0.64      0.65      1209
weighted avg       0.66      0.66      0.66      1209

test score: 0.6617038875103392


## Pipelines Testing per Season

In [18]:
pipe_model = Pipeline([
    ('selector', SelectFromModel(RandomForestClassifier())),
    ('scaler', StandardScaler()),
    # ('normalizer', MinMaxScaler()),
    ('classifier', RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', min_samples_leaf=4, min_samples_split=12, n_estimators=100))
])

pipe_model.fit(X_train, y_train)

Pipeline(steps=[('selector',
                 SelectFromModel(estimator=RandomForestClassifier())),
                ('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=8, max_features='sqrt',
                                        min_samples_leaf=4,
                                        min_samples_split=12))])

In [19]:
# Predict
preds = pipe_model.predict(X_test)
test_score = pipe_model.score(X_test, y_test)

target_names=['home loss', 'home win']

print(classification_report(y_test, preds, target_names=target_names))

print("test score", test_score)


              precision    recall  f1-score   support

   home loss       0.68      0.54      0.60       429
    home win       0.68      0.79      0.73       523

    accuracy                           0.68       952
   macro avg       0.68      0.67      0.67       952
weighted avg       0.68      0.68      0.67       952

test score 0.6785714285714286


## ANN_LSTM Testing per Season

In [15]:
model = Sequential()
model.add(LSTM(50, activation='tanh', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [16]:
arr_X_train = X_train.to_numpy()
arr_X_train = arr_X_train.reshape(arr_X_train.shape[0], arr_X_train.shape[1], 1)

arr_X_test = X_test.to_numpy()
arr_X_test = arr_X_test.reshape(arr_X_test.shape[0], arr_X_test.shape[1], 1)

In [17]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
216/216 [==============================] - 37s 147ms/step - loss: 0.6720 - accuracy: 0.6011
Epoch 2/100
216/216 [==============================] - 32s 150ms/step - loss: 0.6668 - accuracy: 0.6055
Epoch 3/100
216/216 [==============================] - 37s 170ms/step - loss: 0.6622 - accuracy: 0.6151
Epoch 4/100
216/216 [==============================] - 33s 151ms/step - loss: 0.6605 - accuracy: 0.6205
Epoch 5/100
216/216 [==============================] - 34s 157ms/step - loss: 0.6563 - accuracy: 0.6262
Epoch 6/100
216/216 [==============================] - 33s 151ms/step - loss: 0.6532 - accuracy: 0.6276
Epoch 7/100
216/216 [==============================] - 32s 149ms/step - loss: 0.6512 - accuracy: 0.6301
Epoch 8/100
216/216 [==============================] - 32s 150ms/step - loss: 0.6509 - accuracy: 0.6299
Epoch 9/100
216/216 [==============================] - 38s 176ms/step - loss: 0.6488 - accuracy: 0.6314
Epoch 10/100
216/216 [==============================] - 35s 162m

In [18]:
# evaluate the keras model
scores = model.evaluate(arr_X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 65.59
